In [1]:
import wrds
import pandas as pd
import numpy as np
import datetime as dt
from fuzzywuzzy import fuzz
db = wrds.Connection(wrds_username='jhirschi')

Loading library list...
Done


### Initial Exploration of the Datasets available to us from WRDS

In [ ]:
db.list_libraries()

In [ ]:
db.list_tables(library='ibes')

In [ ]:
#a quick way to find out which schemas we actually have access to in WRDS Compustat
libs = db.list_tables(library='ibes')
for lib in libs:
    try:
        db.get_table(library='ibes', table=lib, obs=10)
        print('Success!! This table works: ', lib)
    except Exception:
        pass

In [ ]:
db.list_tables(library='comp')

In [ ]:
#a quick way to find out which schemas we actually have access to in WRDS Compustat
libs = db.list_tables(library='comp')
for lib in libs:
    try:
        db.get_table(library='comp', table=lib, obs=10)
        print('Success!! This table works: ', lib)
    except Exception:
        pass


In [ ]:
#a quick way to find out which schemas we actually have access to in WRDS CRSP
libs = db.list_tables(library='crsp')
for lib in libs:
    try:
        db.get_table(library='crsp', table=lib, obs=10)
        print('Success!! This table works: ', lib)
    except Exception:
        pass


### Begin filtering list of unique tickers/cusips/identifiers

In [28]:
db.get_table(library='crsp', table='stocknames', obs=10)

,permno,permco,namedt,nameenddt,cusip,ncusip,ticker,comnam,hexcd,exchcd,siccd,shrcd,shrcls,st_date,end_date,namedum
0,10000.0,7952.0,1986-01-07,1987-06-11,68391610,68391610,OMFGA,OPTIMUM MANUFACTURING INC,3.0,3.0,3990.0,10.0,A,1986-01-31,1987-06-30,2.0
1,10001.0,7953.0,1986-01-09,1993-11-21,36720410,39040610,GFGC,GREAT FALLS GAS CO,2.0,3.0,4920.0,11.0,None,1986-01-31,2017-08-31,2.0
2,10001.0,7953.0,1993-11-22,2008-02-04,36720410,29274A10,EWST,ENERGY WEST INC,2.0,3.0,4920.0,11.0,None,1986-01-31,2017-08-31,2.0
3,10001.0,7953.0,2008-02-05,2009-08-03,36720410,29274A20,EWST,ENERGY WEST INC,2.0,3.0,4920.0,11.0,None,1986-01-31,2017-08-31,2.0
4,10001.0,7953.0,2009-08-04,2009-12-17,36720410,29269V10,EGAS,ENERGY INC,2.0,3.0,4920.0,11.0,None,1986-01-31,2017-08-31,2.0
5,10001.0,7953.0,2009-12-18,2010-07-08,36720410,29269V10,EGAS,ENERGY INC,2.0,2.0,4925.0,11.0,None,1986-01-31,2017-08-31,2.0
6,10001.0,7953.0,2010-07-09,2017-08-03,36720410,36720410,EGAS,GAS NATURAL INC,2.0,2.0,4925.0,11.0,None,1986-01-31,2017-08-31,2.0
7,10002.0,7954.0,1986-01-10,1993-09-29,05978R10,60740110,MBNC,MOBILE NATIONAL CORP,3.0,3.0,6710.0,10.0,None,1986-01-31,2013-02-28,2.0
8,10002.0,7954.0,1993-09-30,2002-05-14,05978R10,83623410,SABC,SOUTH ALABAMA BANCORPORATION IN,3.0,3.0,6710.0,11.0,None,1986-01-31,2013-02-28,2.0
9,10002.0,7954.0,2002-05-15,2013-02-15,05978R10,05978R10,BTFG,BANCTRUST FINANCIAL GROUP INC,3.0,3.0,6020.0,11.0,None,1986-01-31,2013-02-28,2.0


In [78]:
#make initial pull to get identifying info for the companies in the db that currently exist
#then we group them by the exchange they are on
stk_names = db.raw_sql(''' select permno, permco, cusip, ncusip, ticker, comnam, exchcd, namedt, nameenddt, st_date, end_date from crsp.stocknames where end_date = '2019-12-31' ''')
stk_names[['ticker', 'exchcd']].groupby('exchcd').count()

,ticker
exchcd,
-2.0,8
-1.0,40
0.0,0
1.0,4455
2.0,1458
3.0,5806
4.0,2233
5.0,363
6.0,1


Will only look at companies on exchanges 1 - 5

In [79]:
#add filters to get our selection of companies for which we will need matches
#each row is the most recent version of the company name/ticker/cusip of all the available versions
#this is our "names" dataframe
names = stk_names.loc[(~stk_names['ticker'].isnull())&(stk_names['exchcd'].isin([1,2,3,4,5]))].copy()
names.drop_duplicates(subset=['permno', 'permco', 'cusip', 'ncusip', 'ticker'], keep='first', inplace=True)
names.drop_duplicates(subset=['permno', 'permco', 'cusip'], keep='last', inplace=True)
names

,permno,permco,cusip,ncusip,ticker,comnam,exchcd,namedt,nameenddt,st_date,end_date
0,10026.0,7976.0,46603210,46603210,JJSF,J & J SNACK FOODS CORP,3.0,1986-02-04,2019-12-31,1986-02-28,2019-12-31
12,10028.0,7978.0,29402E10,29402E10,ELA,ENVELA CORP,2.0,2019-12-18,2019-12-31,1986-02-28,2019-12-31
13,10032.0,7980.0,72913210,72913210,PLXS,PLEXUS CORP,3.0,1986-02-05,2019-12-31,1986-02-28,2019-12-31
16,10044.0,7992.0,77467X10,77467X10,RMCF,ROCKY MOUNTAIN CHOC FAC INC NEW,3.0,2015-03-02,2019-12-31,1986-02-28,2019-12-31
21,10051.0,7999.0,41043F20,41043F20,HGR,HANGER ORTHOPEDIC GROUP INC,2.0,1991-05-01,1998-12-14,1986-02-28,2019-12-31
...,...,...,...,...,...,...,...,...,...,...,...
14995,93426.0,53443.0,92835K10,92835K10,VPG,VISHAY PRECISION GROUP INC,1.0,2010-07-07,2019-12-31,2010-07-30,2019-12-31
14996,93427.0,53445.0,G3323L10,G3323L10,FN,FABRINET,1.0,2010-06-25,2019-12-31,2010-06-30,2019-12-31
14997,93429.0,53447.0,12503M10,12503M10,CBOE,C B O E HOLDINGS INC,3.0,2010-06-15,2017-10-17,2010-06-30,2019-12-31
15000,93434.0,53427.0,78513510,78513510,SANW,S & W SEED CO,3.0,2010-06-14,2019-12-31,2010-06-30,2019-12-31


Needed to filter to unique permno/permco/cusip. These identifiers will enable us to match data across the databases, despite the fact that companies relist themselves on different exchanges and change tickers regularly

### Below, will match each company with it's fiscal dates to figure out when its quarterly financial statements are released

##### Get the GVKEY matches, evaluate quality of matches and merge into single table

In [16]:
def get_gvkey_match(cusip):
    df= db.raw_sql('''
    select a.conm, a.gvkey, a.naics, b.cusip as long_cusip, left(b.cusip, 8) as cusip, b.tic from comp.company a 
    left join comp.security b
    on a.gvkey = b.gvkey
    where left(b.cusip, 8) ='{}' '''.format(cusip))
    return df

In [67]:
def evaluate_match(comp, crsp):
    #evaluate precision of cusip match (only one row), ticker match, and company name match
    if len(comp) == 1 and comp['cusip'].iloc[0]==crsp.cusip:
        comp['cusip_score']=1
        comp['ticker_match_score']=fuzz.ratio(comp['tic'].iloc[0], crsp.ticker)
        comp['name_match_score']=fuzz.partial_ratio(comp['conm'].iloc[0], crsp.comnam)
    else:
        comp['cusip_score']=0
    return comp

In [61]:
def get_matches_from_names(names): #the loop function. Requires a "names" dataframe
    match = []
    for ent in names.itertuples():
        temp = get_gvkey_match(ent.cusip)
        temp = evaluate_match(temp, ent)
        match.append(temp)
    match = pd.concat(match)
    names = names.merge(match, how = 'left', on='cusip')
    return names

In [75]:
def clean_up_matches(names):
    names['average_match_score']=names[['ticker_match_score', 'name_match_score']].mean(axis=1)
    names = names.loc[(names['cusip_score']==1) & (names['average_match_score']>75)].copy()
    names = names[['permno', 'permco', 'ncusip', 'cusip', 'ticker', 'comnam', 'tic', 'conm', 'gvkey', 'long_cusip', 'naics', 'exchcd', 'namedt', 'nameenddt', 'st_date', 'end_date', 'cusip_score', 'ticker_match_score', 'name_match_score', 'average_match_score']]
    return names

In [80]:
#get all the matches for the CRSP companies in the "names" dataframe
names = get_matches_from_names(names)
names = clean_up_matches(names)

In [82]:
names[['ticker_match_score', 'name_match_score', 'average_match_score']].mean(axis=0)

ticker_match_score     99.701990
name_match_score       88.224279
average_match_score    93.963134
dtype: float64

In [83]:
names.to_csv('company_names_matched.csv', index=0)

In [86]:
names.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6483 entries, 0 to 7878
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   permno               6483 non-null   float64
 1   permco               6483 non-null   float64
 2   ncusip               6483 non-null   object 
 3   cusip                6483 non-null   object 
 4   ticker               6483 non-null   object 
 5   comnam               6483 non-null   object 
 6   tic                  6483 non-null   object 
 7   conm                 6483 non-null   object 
 8   gvkey                6483 non-null   object 
 9   long_cusip           6483 non-null   object 
 10  naics                6119 non-null   object 
 11  exchcd               6483 non-null   float64
 12  namedt               6483 non-null   object 
 13  nameenddt            6483 non-null   object 
 14  st_date              6483 non-null   object 
 15  end_date             6483 non-null   o

##### Get the quarterly statement dates

In [ ]:
db.describe_table(library='comp', table='fundq').name.tolist()

In [3]:
def get_5_years_of_earnings(gvkey, ticker, long_cusip):
    df = db.raw_sql('''
    select gvkey, tic, cusip as long_cusip, fyr, fqtr, fyearq, datadate, rdq 
    from comp.fundq 
    where tic = '{0}'
    and gvkey= '{1}'
    and cusip = '{2}' 
    and datadate > '2015-06-01'
    order by rdq asc
    '''.format(ticker, gvkey, long_cusip))
    return df

In [100]:
#example of how earnings events appear
get_5_years_of_earnings('012825', 'JJSF', '466032109')

,gvkey,tic,cusip,fyr,fqtr,fyearq,datadate,rdq
0,012825,JJSF,466032109,9.0,3.0,2015.0,2015-06-30,2015-07-27
1,012825,JJSF,466032109,9.0,4.0,2015.0,2015-09-30,2015-11-05
2,012825,JJSF,466032109,9.0,1.0,2016.0,2015-12-31,2016-01-25
3,012825,JJSF,466032109,9.0,2.0,2016.0,2016-03-31,2016-04-25
4,012825,JJSF,466032109,9.0,3.0,2016.0,2016-06-30,2016-07-25
5,012825,JJSF,466032109,9.0,4.0,2016.0,2016-09-30,2016-11-03
6,012825,JJSF,466032109,9.0,1.0,2017.0,2016-12-31,2017-01-23
7,012825,JJSF,466032109,9.0,2.0,2017.0,2017-03-31,2017-04-24
8,012825,JJSF,466032109,9.0,3.0,2017.0,2017-06-30,2017-07-24
9,012825,JJSF,466032109,9.0,4.0,2017.0,2017-09-30,2017-11-09


In [102]:
def build_statements_table(names):
    earnings = []
    temp = names[['gvkey', 'long_cusip', 'tic']].values
    for ent in temp:
        dates = get_5_years_of_earnings(ent[0], ent[2], ent[1])
        earnings.append(dates)
    earnings = pd.concat(earnings)
    return earnings        

In [103]:
earnings = build_statements_table(names)

In [121]:
#consolidate the final earnings table to all needed points for price pulls
final_earnings = names.merge(earnings, how='left', on=['gvkey', 'tic', 'long_cusip'])
final_earnings=final_earnings.loc[(~final_earnings['rdq'].isnull())&(final_earnings['rdq']<='2019-12-20')].copy()
final_earnings.drop(['ncusip', 'tic', 'conm', 'long_cusip', 'namedt', 'nameenddt', 'st_date', 'end_date',
                     'cusip_score', 'ticker_match_score', 'name_match_score', 'average_match_score'], axis=1, inplace=True)
#final_earnings.to_csv('earnings_dates.csv', index=False)
final_earnings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79608 entries, 0 to 118403
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   permno    79608 non-null  float64
 1   permco    79608 non-null  float64
 2   cusip     79608 non-null  object 
 3   ticker    79608 non-null  object 
 4   comnam    79608 non-null  object 
 5   gvkey     79608 non-null  object 
 6   naics     79608 non-null  object 
 7   exchcd    79608 non-null  float64
 8   fyr       79608 non-null  float64
 9   fqtr      79557 non-null  float64
 10  fyearq    79608 non-null  float64
 11  datadate  79608 non-null  object 
 12  rdq       79608 non-null  object 
dtypes: float64(6), object(7)
memory usage: 8.5+ MB


In [2]:
names = pd.read_csv('company_names_matched.csv')
final_earnings=pd.read_csv('earnings_dates.csv')

### Below will perform analysis on all companies in our list so far to determine what a threshold of a "big move" around earnings time is. We will begin building a labeled dataset as we engineer features to input into our final model

##### Determining threshold for what moves at earnings time are of interest

We don't want to look at moves that are too big (could be resulting from a one time corporate action, a price split/reverse split, etc). We also don't want to look at moves that are big % wise but are too small nominally to be of much use in an options play. Looking at a sweet spot where there is atleast a certain size % change and also a big enough $ change, but not so big as to be a result of something other than what is a pure result of the earnings results

In [ ]:
db.describe_table(library='crsp', table='dsf')

In [11]:
def get_20_trading_days(date,permno, cusip):
    date = dt.datetime.strptime(date, '%Y-%m-%d').date()
    st_dt = date+dt.timedelta(days=-14)
    end_dt = date+dt.timedelta(days=14)
    df = db.raw_sql('''
    select cusip, permno, date, prc, vol/1000 as vol, shrout, cfacpr, cfacshr, numtrd, 
    (prc * cfacpr) as adj_prc, (shrout *cfacshr) as adj_shares, (vol/(shrout*cfacshr*1000)) as pct_traded,
    case when prc != (prc*cfacpr) then 1 else 0 end as adj
    from crsp.dsf
    where date >= '{0}'
    and date <= '{1}'
    and permno = {2}
    and cusip = '{3}'
    and cfacshr > 0
    and shrout > 0
    '''.format(st_dt, end_dt, permno, cusip))
    return df
    

In [ ]:
#taking a random sample of 1000 earnings events to test different nominal and percentage changes for my thresholds
#test for daily returns, 2 day returns, 5 day returns, 10 day returns, and 20 day returns. 
#As well as volatility over the entire period

#graph some histograms of the distributions of your samples for volatility, and for returns, and nominal changes. 
#

In [7]:
trad = get_20_trading_days('2019-10-30' , 14593, '03783310')

In [4]:
def evaluate_earnings(date, permno, cusip, pct_thresh=0.07, nom_thresh=6, cum_pct_thresh_1=0.12, 
                      cum_nom_thresh_1=10, cum_pct_thresh_2=0.12, cum_nom_thresh_2=10 ):
    df = get_20_trading_days(date, permno, cusip)
    if df.empty:
        return ['Error']
    
    days=len(df['adj_prc'])
    if days<=7:
        return ['Error']
    
    #evaluates the pct and nom diffs
    try:
        pct_diff = df['adj_prc'].pct_change()
        nom_diff = df['adj_prc'].diff()
        max_pct_diff = pct_diff.max()
        max_nom_diff=nom_diff.max()
    except Exception:
        return ['Error']
    
    #evaluates the cumulate pct and nom diffs
    start, end = 0, 4
    cum_pct_diff = []
    cum_nom_diff = []
    while end< days:
        cum_pct_diff.append(pct_diff.iloc[start:end].sum())
        cum_nom_diff.append(nom_diff.iloc[start:end].sum())
        start+=1
        end+=1
    max_cum_pct_diff = max(cum_pct_diff)
    max_cum_nom_diff = max(cum_nom_diff)
    
    #evaluates the cumulate pct and nom diffs level 2
    start, end = 0, 7
    cum_pct_diff_2 = []
    cum_nom_diff_2 = []
    while end< days:
        cum_pct_diff_2.append(pct_diff.iloc[start:end].sum())
        cum_nom_diff_2.append(nom_diff.iloc[start:end].sum())
        start+=1
        end+=1
    max_cum_pct_diff_2 = max(cum_pct_diff_2)
    max_cum_nom_diff_2 = max(cum_nom_diff_2)
    
    #boolean checks
    move_pct = max_pct_diff > pct_thresh
    move_cum_pct_1 = max_cum_pct_diff > cum_pct_thresh_1
    move_cum_pct_2 = max_cum_pct_diff_2 > cum_pct_thresh_2
    
    move_nom = max_nom_diff > nom_thresh
    move_cum_nom_1 = max_cum_nom_diff > cum_nom_thresh_1
    move_cum_nom_2 = max_cum_nom_diff_2 > cum_nom_thresh_2
    
    return move_pct, move_nom, move_cum_pct_1, move_cum_nom_1, move_cum_pct_2, move_cum_nom_2, 

In [50]:
test = get_20_trading_days('2018-04-23', 56641.0, '02927U20' )

In [77]:
sample = final_earnings.sample(n=600)
sample

,permno,permco,cusip,ticker,comnam,gvkey,naics,exchcd,fyr,fqtr,fyearq,datadate,rdq
57923,85337.0,32078.0,45646310,IBA,INDUSTRIAS BACHOCO S A DE C V,221545,311615,1.0,12.0,3.0,2015.0,2015-09-30,2015-10-22
44471,76076.0,10486.0,17275R10,CSCO,CISCO SYSTEMS INC,20779,334210,3.0,7.0,1.0,2019.0,2018-10-31,2018-11-14
12700,14521.0,54772.0,89628810,TNET,TRINET GROUP INC,140988,518210,1.0,12.0,2.0,2018.0,2018-06-30,2018-07-30
69655,90427.0,45732.0,88268110,TXRH,TEXAS ROADHOUSE INC,160376,722511,3.0,12.0,4.0,2017.0,2017-12-31,2018-02-20
31169,18426.0,56666.0,Y2067610,DSSI,DIAMOND S SHIPPING INC,34956,483111,1.0,12.0,4.0,2018.0,2018-12-31,2019-03-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74295,91742.0,51660.0,83558L20,SNOA,SONOMA PHARMACEUTICALS INC,175424,325412,3.0,3.0,1.0,2015.0,2015-06-30,2015-07-30
43775,75416.0,21995.0,74971D10,RPT,R P T REALTY,15111,531120,1.0,12.0,2.0,2015.0,2015-06-30,2015-07-21
27503,17251.0,56220.0,65441V10,NINE,NINE ENERGY SERVICES INC,31507,213112,1.0,12.0,3.0,2018.0,2018-09-30,2018-11-12
51253,80498.0,13092.0,03076K10,ABCB,AMERIS BANCORP,30222,522110,3.0,12.0,4.0,2016.0,2016-12-31,2017-01-20


In [5]:
def get_aggregate_earnings_evals(df, pct_thresh=0.07, nom_thresh=6, cum_pct_thresh_1=0.12, 
                      cum_nom_thresh_1=10, cum_pct_thresh_2=0.12, cum_nom_thresh_2=10 ):
    evals = []
    counter = 0
    keys=['move_pct', 'move_nom', 'move_cum_pct_1', 'move_cum_nom_1', 'move_cum_pct_2', 'move_cum_nom_2']
    try:
        for row in df.itertuples():
            #print(counter)
            temp = evaluate_earnings(row.rdq, row.permno, row.cusip, pct_thresh=pct_thresh, nom_thresh=nom_thresh, 
                                     cum_pct_thresh_1=cum_pct_thresh_1, cum_pct_thresh_2=cum_pct_thresh_2, 
                                     cum_nom_thresh_2=cum_nom_thresh_2)
            if 'Error' in temp:
                evals.append({'rdq':row.rdq, 'permno':row.permno, 'cusip':row.cusip, 'Error':True})
            else:
                temp = [1 if x == True else 0 for x in temp]
                temp = {key:val for key, val in zip(keys, temp)}
                temp = {**temp, **{'rdq':row.rdq, 'permno':row.permno, 'cusip':row.cusip, 'Error':False}}
                evals.append(temp)
            counter+=1
            if counter%100 ==0:
                print(counter)    
        evals = pd.DataFrame(evals)    
        return evals
    
    except Exception:
        print('It broke here:', counter)
        return pd.DataFrame(evals)
    

In [81]:
eval_sample = get_aggregate_earnings_evals(sample)

100
200
300
400
500
600


In [82]:
eval_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   move_pct        574 non-null    float64
 1   move_nom        574 non-null    float64
 2   move_cum_pct_1  574 non-null    float64
 3   move_cum_nom_1  574 non-null    float64
 4   move_cum_pct_2  574 non-null    float64
 5   move_cum_nom_2  574 non-null    float64
 6   rdq             600 non-null    object 
 7   permno          600 non-null    float64
 8   cusip           600 non-null    object 
 9   Error           600 non-null    bool   
dtypes: bool(1), float64(7), object(2)
memory usage: 42.9+ KB


In [93]:
temp_eval=eval_sample.loc[~eval_sample['move_nom'].isnull()].copy()
temp_eval.to_csv('sample_evals.csv', index=False)

##### test for big moves around those dates for the 10 trading days before/after the announcement for the last 5 years

In [ ]:
final_eval = get_aggregate_earnings_evals(final_earnings)

In [14]:
final_eval.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61400 entries, 0 to 61399
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   move_pct        59080 non-null  float64
 1   move_nom        59080 non-null  float64
 2   move_cum_pct_1  59080 non-null  float64
 3   move_cum_nom_1  59080 non-null  float64
 4   move_cum_pct_2  59080 non-null  float64
 5   move_cum_nom_2  59080 non-null  float64
 6   rdq             61400 non-null  object 
 7   permno          61400 non-null  float64
 8   cusip           61400 non-null  object 
 9   Error           61400 non-null  bool   
dtypes: bool(1), float64(7), object(2)
memory usage: 4.3+ MB


In [15]:
final_eval.to_csv('final_eval_pt1.csv', index=False)

In [15]:
part_2 = final_earnings.iloc[61400:].copy()
part_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9448 entries, 61400 to 70847
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   permno    9448 non-null   float64
 1   permco    9448 non-null   float64
 2   cusip     9448 non-null   object 
 3   ticker    9448 non-null   object 
 4   comnam    9448 non-null   object 
 5   gvkey     9448 non-null   int64  
 6   naics     9448 non-null   int64  
 7   exchcd    9448 non-null   float64
 8   fyr       9448 non-null   float64
 9   fqtr      9445 non-null   float64
 10  fyearq    9448 non-null   float64
 11  datadate  9448 non-null   object 
 12  rdq       9448 non-null   object 
dtypes: float64(6), int64(2), object(5)
memory usage: 959.7+ KB


In [ ]:
final_eval_pt2 = get_aggregate_earnings_evals(part_2)

In [17]:
final_eval_pt2.to_csv('final_eval_pt2.csv', index=False)

In [18]:
final_eval_pt1 = pd.read_csv('final_eval_pt1.csv')

In [19]:
final_evals = pd.concat([final_eval_pt1, final_eval_pt2])

In [20]:
final_evals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70848 entries, 0 to 9447
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   move_pct        68449 non-null  float64
 1   move_nom        68449 non-null  float64
 2   move_cum_pct_1  68449 non-null  float64
 3   move_cum_nom_1  68449 non-null  float64
 4   move_cum_pct_2  68449 non-null  float64
 5   move_cum_nom_2  68449 non-null  float64
 6   rdq             70848 non-null  object 
 7   permno          70848 non-null  float64
 8   cusip           70848 non-null  object 
 9   Error           70848 non-null  bool   
dtypes: bool(1), float64(7), object(2)
memory usage: 5.5+ MB


In [22]:
1-(68449/70848)

0.03386122402890701

In [23]:
final_evals.to_csv('total_final_evals.csv', index=False)

In [27]:
final_evals.reset_index(drop=True, inplace=True)

##### Repeat for all companies and compile a table with yes/no labels for each period. 

The final output will be ticker/cusip/permno/company_name and then date and label. The final determinant for whether or not there was a "big move" is if >=4/6 of the thresholds have been met

In [29]:
final_evals['thresholds_passed']=final_evals[['move_pct', 'move_nom', 'move_cum_pct_1', 'move_cum_nom_1', 'move_cum_pct_2', 'move_cum_nom_2']].sum(axis=1)

In [30]:
final_evals['Earnings_Move']=0
final_evals.loc[final_evals['thresholds_passed']>= 4, 'Earnings_Move']=1

In [32]:
final_evals[['permno', 'Earnings_Move']].groupby('Earnings_Move').count()

,permno
Earnings_Move,
0,68441
1,2407


In [33]:
2407/68449

0.035164867273444464

In [ ]:
#combine everything into final dataset
#write to csv and commit to github

In [34]:
final_evals = final_evals.loc[final_evals['Error']==False]
final_evals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68449 entries, 0 to 70847
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   move_pct           68449 non-null  float64
 1   move_nom           68449 non-null  float64
 2   move_cum_pct_1     68449 non-null  float64
 3   move_cum_nom_1     68449 non-null  float64
 4   move_cum_pct_2     68449 non-null  float64
 5   move_cum_nom_2     68449 non-null  float64
 6   rdq                68449 non-null  object 
 7   permno             68449 non-null  float64
 8   cusip              68449 non-null  object 
 9   Error              68449 non-null  bool   
 10  thresholds_passed  68449 non-null  float64
 11  Earnings_Move      68449 non-null  int64  
dtypes: bool(1), float64(8), int64(1), object(2)
memory usage: 6.3+ MB


In [35]:
final_evals.to_csv('total_final_evals.csv', index=False)

In [2]:
fin = pd.read_csv('final_data.csv')

In [3]:
fin

,ticker,permno,rdq,datadate,fqtr,fyearq,exchcd,exch,exch_ticker,Company_Name,...,move_pct,move_nom,move_cum_pct_1,move_cum_nom_1,move_cum_pct_2,move_cum_nom_2,cusip,Error,thresholds_passed,Earnings_Move
0,JJSF,10026,2015-07-27,2015-06-30,3,2015,3,NASDAQ,NASDAQ:JJSF,J & J Snack Foods Corp.,...,0.0,0.0,0.0,0.0,0.0,0.0,46603210,False,0.0,0
1,JJSF,10026,2015-11-05,2015-09-30,4,2015,3,NASDAQ,NASDAQ:JJSF,J & J Snack Foods Corp.,...,0.0,0.0,0.0,0.0,0.0,0.0,46603210,False,0.0,0
2,JJSF,10026,2016-01-25,2015-12-31,1,2016,3,NASDAQ,NASDAQ:JJSF,J & J Snack Foods Corp.,...,0.0,0.0,0.0,0.0,0.0,0.0,46603210,False,0.0,0
3,JJSF,10026,2016-04-25,2016-03-31,2,2016,3,NASDAQ,NASDAQ:JJSF,J & J Snack Foods Corp.,...,0.0,0.0,0.0,0.0,0.0,0.0,46603210,False,0.0,0
4,JJSF,10026,2016-07-25,2016-06-30,3,2016,3,NASDAQ,NASDAQ:JJSF,J & J Snack Foods Corp.,...,0.0,0.0,0.0,0.0,0.0,0.0,46603210,False,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64478,IPI,92621,2015-07-28,2015-06-30,2,2015,1,NYSE,NYSE:IPI,"Intrepid Potash, Inc.",...,0.0,0.0,0.0,0.0,0.0,0.0,46121Y10,False,0.0,0
64479,IPI,92621,2015-10-27,2015-09-30,3,2015,1,NYSE,NYSE:IPI,"Intrepid Potash, Inc.",...,0.0,0.0,0.0,0.0,0.0,0.0,46121Y10,False,0.0,0
64480,IPI,92621,2016-02-29,2015-12-31,4,2015,1,NYSE,NYSE:IPI,"Intrepid Potash, Inc.",...,1.0,0.0,1.0,0.0,1.0,0.0,46121Y10,False,3.0,0
64481,IPI,92621,2016-05-09,2016-03-31,1,2016,1,NYSE,NYSE:IPI,"Intrepid Potash, Inc.",...,1.0,0.0,1.0,0.0,1.0,0.0,46121Y10,False,3.0,0


In [4]:
fin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64483 entries, 0 to 64482
Data columns (total 67 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ticker                       64465 non-null  object 
 1   permno                       64483 non-null  int64  
 2   rdq                          64483 non-null  object 
 3   datadate                     64483 non-null  object 
 4   fqtr                         64483 non-null  int64  
 5   fyearq                       64483 non-null  int64  
 6   exchcd                       64483 non-null  int64  
 7   exch                         64483 non-null  object 
 8   exch_ticker                  64483 non-null  object 
 9   Company_Name                 64483 non-null  object 
 10  IQ_TOTAL_DEBT_EQUITY         61715 non-null  float64
 11  IQ_TOTAL_DEBT_CAPITAL        64103 non-null  float64
 12  IQ_EBITDA_INT                48154 non-null  float64
 13  IQ_EBITDA_CAPEX_